In [1]:
import numpy as np
import pandas as pd
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/submission.csv')

In [6]:
#케라스가 내 모델에 맞추기 위해 사용할 수 있는 one-hot 벡터 배열로 바꾸기
from keras.utils.np_utils import to_categorical
#입력이미지의 shape 이 (batch_size, width, height, n_channels)로 넣어줘야함
X_train = (train[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 28, 28, 1)
print(X_train.shape)
#레이블 준비하기
y_train = to_categorical(train['digit'].values)
print(X_train[0].shape)
#print(len(y_train))
print(y_train.shape)

(2048, 28, 28, 1)
(28, 28, 1)
(2048, 10)


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 아마 성능 향상의 가장 큰 요인, 데이터 증강
# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Dropout, MaxPool2D,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add
)
#from keras.layers import LeakyReLU


model = Sequential() #클래스 객체 생성
model.add(Conv2D(32, kernel_size = 3, input_shape = (28, 28, 1)))  #26 * 26 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 3)) # 24 * 24 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu')) #12 * 12 * 32
model.add(Dropout(0.4)) 

model.add(Conv2D(64, kernel_size = 3))# 10 * 10 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 3)) # 8 * 8 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu')) # 4 * 4 * 64
model.add(Dropout(0.4)) 

model.add(Conv2D(128, kernel_size = 4)) # 1 * 1 * 128
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
____

In [4]:
# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [6]:
# 좀 더 빨리 하기 위함
from tensorflow.keras.callbacks import LearningRateScheduler
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [21]:
from sklearn.model_selection import train_test_split
epochs = 45
# Train-Test를 9:1로 분리
X_train2, X_val2, y_train2, y_val2 = train_test_split(
    X_train, y_train, test_size = 0.1)

#fit 메서드로 트레이닝 시작
hist = model.fit(
    X_train2, y_train2, batch_size = 20,
    epochs=epochs, 
    
    #steps_per_epoch=X_train2.shape[0]//20,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer],
    verbose=1
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

Train on 1843 samples, validate on 205 samples
Epoch 1/45
1843/1843 [==============================] - 7s 4ms/sample - loss: 0.8711 - acc: 0.7254 - val_loss: 0.5273 - val_acc: 0.8732
Epoch 2/45
1843/1843 [==============================] - 9s 5ms/sample - loss: 0.6014 - acc: 0.7878 - val_loss: 0.4172 - val_acc: 0.8683
Epoch 3/45
1843/1843 [==============================] - 9s 5ms/sample - loss: 0.5047 - acc: 0.8199 - val_loss: 0.5709 - val_acc: 0.8293
Epoch 4/45
1843/1843 [==============================] - 8s 4ms/sample - loss: 0.3878 - acc: 0.8681 - val_loss: 0.3881 - val_acc: 0.9024
Epoch 5/45
1843/1843 [==============================] - 8s 4ms/sample - loss: 0.3947 - acc: 0.8540 - val_loss: 0.3566 - val_acc: 0.9122
Epoch 6/45
1843/1843 [==============================] - 8s 4ms/sample - loss: 0.2958 - acc: 0.8953 - val_loss: 0.3780 - val_acc: 0.8878
Epoch 7/45
1843/1843 [==============================] - 8s 4ms/sample - loss: 0.2758 - acc: 0.9088 - val_loss: 0.3884 - val_acc: 0.8927
E

In [ ]:
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=1
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

In [ ]:
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=0
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

In [ ]:
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=0
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

In [ ]:
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=0
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

In [ ]:
#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=0
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

In [ ]:
# Parameter 및 모델 구조 저장
model.save_weights(f'params.h5')
    
model_json = model.to_json()
with open(f"mode_6Times.json", "w") as json_file : 
    json_file.write(model_json)

In [ ]:
# 예측 진행
X_test = (test[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 28, 28, 1)
#results = model.predict(X_test)
submission['digit']=np.argmax(model.predict(X_test),axis =1)

In [ ]:
#submission.digit = results
submission.to_csv('model_6Times.csv', index=False)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Dropout, MaxPool2D,
    Flatten, Dense, Input, Concatenate, LeakyReLU, Add
)
#from keras.layers import LeakyReLU


model = Sequential() #클래스 객체 생성
model.add(Conv2D(32, kernel_size = 3, input_shape = (28, 28, 1), kernel_initializer='he_normal'))  #26 * 26 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 3,kernel_initializer='he_normal')) # 24 * 24 * 32
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu', kernel_initializer='he_normal')) #12 * 12 * 32
model.add(Dropout(0.4)) 

model.add(Conv2D(64, kernel_size = 3,kernel_initializer='he_normal'))# 10 * 10 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 3,kernel_initializer='he_normal')) # 8 * 8 * 64
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu',kernel_initializer='he_normal')) # 4 * 4 * 64
model.add(Dropout(0.4)) 

model.add(Conv2D(128, kernel_size = 4,kernel_initializer='he_normal')) # 1 * 1 * 128
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

In [17]:
# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [18]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 12, 12, 32)        25632     
_________________________________________________________________
dropout_12 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 10, 10, 64)       

In [19]:
# 좀 더 빨리 하기 위함
from tensorflow.keras.callbacks import LearningRateScheduler
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [20]:
from sklearn.model_selection import train_test_split
epochs = 45
# Train-Test를 9:1로 분리
X_train2, X_val2, y_train2, y_val2 = train_test_split(
    X_train, y_train, test_size = 0.1)

#fit 메서드로 트레이닝 시작
hist = model.fit_generator(
    datagen.flow(X_train2, y_train2, batch_size=32),
    epochs=epochs, 
    steps_per_epoch=X_train2.shape[0]//32,
    validation_data=(X_val2, y_val2), 
    callbacks=[annealer], 
    verbose=0
)
print(
    f"CNN: Epochs={epochs:d}, " +
    f"Train accuracy={max(hist.history['acc']):.5f}, " +
    f"Validation accuracy={max(hist.history['val_acc']):.5f}"
)

CNN: Epochs=45, Train accuracy=0.72281, Validation accuracy=0.80976
